In [1]:
import pandas as pd

# Read and format dates and numbers correctly

In [8]:
df = pd.read_csv('data.csv', decimal=',')
df["title"]= pd.to_datetime(df["title"], format="%d.%m.%Y")

df.loc[df['name'].str.contains("1 kk"), 'name'] = 'euribor_1_month'
df.loc[df['name'].str.contains("3 kk"), 'name'] = 'euribor_3_month'
df.loc[df['name'].str.contains("6 kk"), 'name'] = 'euribor_6_month'
df.loc[df['name'].str.contains("12 kk"), 'name'] = 'euribor_12_month'
df = df.drop(df[df['name'].str.contains("Eonia")].index)

df.dropna(inplace=True)

display(df)
display(df.info())

,title,name,value
0,2023-04-25,euribor_1_month,3.024
1,2023-04-25,euribor_3_month,3.268
2,2023-04-25,euribor_6_month,3.635
3,2023-04-25,euribor_12_month,3.900
4,2023-04-24,euribor_1_month,3.031
...,...,...,...
31189,1999-01-05,euribor_12_month,3.187
31191,1999-01-04,euribor_1_month,3.242
31192,1999-01-04,euribor_3_month,3.234
31193,1999-01-04,euribor_6_month,3.223


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24968 entries, 0 to 31194
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   24968 non-null  datetime64[ns]
 1   name    24968 non-null  object        
 2   value   24968 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 780.2+ KB


None

# Transform data to column operational format

In [9]:
euribor_12 = df.loc[df['name'] == 'euribor_12_month',   ['title', 'value']].rename(columns = {'value':'euribor_12_month'})
euribor_6 = df.loc[df['name'] == 'euribor_6_month',     ['title', 'value']].rename(columns = {'value':'euribor_6_month'})
euribor_3 = df.loc[df['name'] == 'euribor_3_month',     ['title', 'value']].rename(columns = {'value':'euribor_3_month'})
euribor_1 = df.loc[df['name'] == 'euribor_1_month',     ['title', 'value']].rename(columns = {'value':'euribor_1_month'})

df = pd.merge(euribor_12, euribor_6, how="inner", left_on='title', right_on='title')
df = pd.merge(df, euribor_3, how="inner", left_on='title', right_on='title')
df = pd.merge(df, euribor_1, how="inner", left_on='title', right_on='title')

df = df.rename(columns = {'title':'date'})
df = df.set_index('date')
df = df.resample('D')
df = df.ffill()

display(df)
display(df.info())

,euribor_12_month,euribor_6_month,euribor_3_month,euribor_1_month
date,,,,
1999-01-04,3.209,3.223,3.234,3.242
1999-01-05,3.187,3.204,3.222,3.233
1999-01-06,3.176,3.197,3.214,3.226
1999-01-07,3.158,3.179,3.206,3.224
1999-01-08,3.139,3.166,3.196,3.217
...,...,...,...,...
2023-04-21,3.854,3.601,3.261,2.956
2023-04-22,3.854,3.601,3.261,2.956
2023-04-23,3.854,3.601,3.261,2.956


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8878 entries, 1999-01-04 to 2023-04-25
Freq: D
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   euribor_12_month  8878 non-null   float64
 1   euribor_6_month   8878 non-null   float64
 2   euribor_3_month   8878 non-null   float64
 3   euribor_1_month   8878 non-null   float64
dtypes: float64(4)
memory usage: 346.8 KB


None

In [11]:
df.to_csv('euribor_rates.csv')